# Predicting Formula 1 Fastest Lap Speed & Time

**Group:** V10FTW  
**Members:** Oskar Floeck s3725028 & Connor Hutchinson s3544152

## Table of contents

* [Source and Description](#desc)
* [Goals and Objectives](#goals)
* [Data Preparation](#data)
* [Data Exploration](#explore)
* [Statistical Modelling & Performance Evaluation](#model)
* [Summary & Conclusion](#conc)

## Source and Description <a name="desc">

### Data Source

Formula 1 Raw Data: http://ergast.com/mrd (Ergast, 2020)  
Track Weather Data: https://www.motorsport-total.com/formel-1/ergebnisse (Motorsport-total, 2020)

Given the way in which weather data was provided by the website above, a script was written to automatically grab neccesary information and store within `track_weather.csv` for analysis in this report. The full script can be [viewed here](https://github.com/floeck/f1-weather-analysis/blob/master/notebooks/scrape-weather.ipynb).

* `races.csv` and `results.csv` obtained from Ergast: Contain all the relevant track data, such as fastest lap speeds, times and date.

* `track_weather.csv` from motorsport-total: Contain track weather information, such as humidity etc.

For the purposes of the report and due to track weather scraping limitations, the following tracks will be used in the analysis from the year 2007 to 2019. Details of these limitations are viewable on the script.

<table>
<tr>
</tr>
<tr>
<td>
    
* Albert Park, Australia
* Sepang, Malaysia
* Sachir, Bahrain
* Catalunya, Spain
* De Monaco, Europe 
* Montreal, Canada
* Magny Cours, France
* Silverstone, Britain
* Valencia, Germany
* Hungaroring, Hungary 
    
</td>
<td>
    
* Istanbul, Turkey
* Monza, Italy
* De Spa, Belgium
* Fuji, Japan
* Shanghai, China
* Interlagos, Brazil
* Yas Marina, Abu Dhabi
* Indianapolis, America
* Austin, America
* Hermanos, Mexico
    
</td>
</tr>
</table>

### Descriptive Features

Descriptions of data for `races.csv`

| feature | type  | units  | desc  |
|---|---|---|---|
| raceId  | ordinal | unknown | ID of the race |
| name | categorical | unknown  | Name of the race |
| date | date | yyyy-mm-dd | Date of race |

Descriptions of data for `results.csv`

| feature | type  | units  | desc  |
|---|---|---|---|
| raceId  | ordinal | unknown | ID of the race |
| fastestLapTime | positive real number | milliseconds  | Fastest lap time |
| fastestLapSpeed | positive real number | kph | Fastest average lap speed |

Descriptions of data for `track_weather.csv`

| feature | type  | units  | desc  |
|---|---|---|---|
| track  | categorical | unknown | Track name |
| date | date | yyyy-mm-dd | Date of race |
| local_time | time | hh:mm:ss | Time of race |
| weather | categorical | unknown | Track conditions |
| temp | positive real number | celcius | Ambient temp |
| track_temp | positive real number | celcius | Track temp |
| humidity | positive real number | % | Ambient humidity |
| air_pressure | positive real number | mBar | Ambient air pressure |
| wind_speed | positive real number | m/s | Track wind speed |
| wind_direction | categorical | direction | Track wind direction |


### Target Feature
The target feature is `fastestLapSpeed` and `fastestLapTime`, both of which are continuous numerical features.

## Goals and Objectives <a name="goals">

The primary goal of the report is to investigate whether or not we can accurately and reliably predict Formula 1 performance in the domain of fastest lap time and average speed, given the weather and track data we have collected via multiple linear regression.

## Data Preparation <a name="data">

### Preliminaries

First, import all necessary modules required for the analysis. A helper function `to_milliseconds` has been written to assist in converting lap times to milliseconds, so that they can be more appropriately analysed.

In [1]:
# Module imports
import warnings
import pandas as pd 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Disable warnings and allow large columns
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

# Function that allows to convert time
def to_milliseconds(string):
    string = string.replace(".", ":").split(":")
    minutes = int(string[0])
    seconds = int(string[1])
    milliseconds = int(string[2])
    return (minutes * 60000) + (seconds * 1000) + milliseconds

Here we read in the data directly from the repository site.

In [2]:
# Read in data to notebook
races = pd.read_csv('../data/formula-1/races.csv', sep=',')
results = pd.read_csv('../data/formula-1/results.csv', sep=',')
track_weather = pd.read_csv('../data/weather/track_weather.csv', sep=',')

# Or if running notebook outside repository...
#races = pd.read_csv('https://raw.githubusercontent.com/floeck/f1-weather-analysis/master/data/formula-1/races.csv', sep = ',')
#results = pd.read_csv('https://raw.githubusercontent.com/floeck/f1-weather-analysis/master/data/formula-1/results.csv', sep=',')
#track_weather = pd.read_csv('https://raw.githubusercontent.com/floeck/f1-weather-analysis/master/data/weather/track_weather.csv', sep=',')

Dataset: `races.csv`

In [3]:
races.sample(5, random_state=999)

,raceId,year,round,circuitId,name,date,time,url
685,686,1967,8,48,Canadian Grand Prix,1967-08-27,\N,http://en.wikipedia.org/wiki/1967_Canadian_Gra...
117,118,2003,11,9,British Grand Prix,2003-07-20,\N,http://en.wikipedia.org/wiki/2003_British_Gran...
160,161,2000,4,9,British Grand Prix,2000-04-23,\N,http://en.wikipedia.org/wiki/2000_British_Gran...
729,730,1962,2,6,Monaco Grand Prix,1962-06-03,\N,http://en.wikipedia.org/wiki/1962_Monaco_Grand...
94,95,2004,6,6,Monaco Grand Prix,2004-05-23,\N,http://en.wikipedia.org/wiki/2004_Monaco_Grand...


Dataset: `results.csv`

In [4]:
results.sample(5, random_state=999)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
20959,20962,848,5,205,20,19,19,19,19,0.0,55,\N,\N,44,19,1:45.055,185.697,12
24036,24042,1001,807,4,27,20,\N,R,20,0.0,0,\N,\N,\N,0,\N,\N,3
6593,6594,308,138,25,3,11,\N,R,20,0.0,35,\N,\N,\N,\N,\N,\N,20
5388,5389,266,50,22,6,12,3,3,3,4.0,77,+1:10.329,6550514,\N,\N,\N,\N,1
1691,1692,98,13,15,12,15,\N,R,18,0.0,0,\N,\N,\N,\N,\N,\N,4


Dataset: `track_weather.csv`

In [5]:
track_weather.sample(5, random_state=999)

,track,date,local_time,weather,temp,track_temp,humidity,air_pressure,wind_speed,wind_direction
96,austin,2012-11-18,13:00:00,sunny,24.0,32.0,29.0,1004.0,5.0,south
245,abu-dhabi,2018-11-25,17:10:00,"clear, short showers",29.0,31.0,55.0,1014.0,3.5,turning southeast
68,montreal,2010-06-13,12:00:00,slightly cloudy,26.0,38.0,42.0,1011.0,3.0,northeast
162,istanbul,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,indianapolis,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Cleaning and Transformation

Now that all the data is imported, we can begin cleaning and remove unneccesary information.

In [6]:
# Drop unneccesary columns
races = races.drop(columns = ['year', 'circuitId', 'round', 'time', 'url'])
results = results.drop(columns = ['resultId', 'driverId', 'constructorId', 'number', 'grid', 'position',
                                  'positionText', 'positionOrder', 'points', 'laps', 'fastestLap', 'time',
                                  'milliseconds', 'fastestLap', 'rank', 'statusId'])
track_weather = track_weather.drop(columns = 'local_time')

# Update problem data in columns
results = results.replace(r'\\N','null', regex=True)

# Query only those with races finishes
results = results.loc[results['fastestLapTime'] != 'null']

# Convert fastest lap speed to float
results['fastestLapSpeed'] = results['fastestLapSpeed'].astype(float)

# Convert fastest time to milliseconds
results['fastestLapTime'] = results['fastestLapTime'].apply(to_milliseconds)

# Group by track and date, then average fastest lap and top speed
races_results = races.merge(results, on = 'raceId').groupby(['raceId', 'name', 'date']).mean()
df = races_results.merge(track_weather, on = 'date')

# Add rain column
rain_desc = ["rain", "shower", "drizzle", "wet"]
df['rain'] = np.where(df['weather'].str.contains('|'.join(rain_desc)), 'wet', 'dry')

# Add relevant categorical columns
df['year'] = df['date'].str[:4].astype(int).astype('category')
df['track'] = df['track'].astype('category')
df['rain'] = df['rain'].astype('category')

# Round & export dataframe to csv
df = df.round(3)
df.to_csv('../data/V10FTW_Data.csv')

Now lets have a look to see our dataframe features match the description outlined earlier in the report.

In [7]:
print(f"Shape of the dataset is {df.shape} \n")
print(df.dtypes)

Shape of the dataset is (215, 13) 

date                 object
fastestLapTime      float64
fastestLapSpeed     float64
track              category
weather              object
temp                float64
track_temp          float64
humidity            float64
air_pressure        float64
wind_speed          float64
wind_direction       object
rain               category
year               category
dtype: object


### Check for Missing Values

In [8]:
print(f"\nNumber of missing values for each feature:")
print(df.isnull().sum())


Number of missing values for each feature:
date               0
fastestLapTime     0
fastestLapSpeed    0
track              0
weather            0
temp               0
track_temp         0
humidity           0
air_pressure       0
wind_speed         0
wind_direction     0
rain               0
year               0
dtype: int64


### Summary Statistics

In [9]:
from IPython.display import display, HTML
display(HTML('<b>Table 1: Summary of continuous features</b>'))
df.describe(include='float64')

,fastestLapTime,fastestLapSpeed,temp,track_temp,humidity,air_pressure,wind_speed
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000
mean,92600.561456,201.440167,24.074419,35.190698,51.269767,987.548837,2.922326
std,11098.117244,19.865508,5.040402,9.345193,16.970061,72.822125,1.644096
min,72460.200000,144.650000,12.000000,14.000000,5.000000,99.000000,0.600000
25%,83092.395500,191.745000,20.500000,29.000000,40.000000,991.000000,1.500000
50%,92599.222000,200.785000,24.000000,34.000000,50.000000,1003.000000,2.500000
75%,100946.418000,213.594500,27.500000,42.000000,61.000000,1012.000000,4.000000
max,121779.435000,248.377000,37.000000,60.000000,98.000000,1025.000000,8.000000


In [10]:
display(HTML('<b>Table 2: Summary of categorical features</b>'))
df.describe(include=['category', 'object'])

,date,track,weather,wind_direction,rain,year
count,215,215,215,215,215,215
unique,206,20,34,16,2,13
top,2016-10-23,interlagos,sunny,south,dry,2016
freq,2,13,83,31,182,18


### Continuous Features

Testing for potential multicollinearity problems that will effect our models performance.

In [11]:
df.corr(method = 'spearman')

,fastestLapTime,fastestLapSpeed,temp,track_temp,humidity,air_pressure,wind_speed
fastestLapTime,1.000000,0.001423,-0.052734,-0.356947,0.160068,0.138620,0.187672
fastestLapSpeed,0.001423,1.000000,-0.071805,-0.003104,-0.078459,-0.215872,0.053095
temp,-0.052734,-0.071805,1.000000,0.616918,-0.318113,0.073936,-0.116751
track_temp,-0.356947,-0.003104,0.616918,1.000000,-0.472892,-0.082002,-0.205659
humidity,0.160068,-0.078459,-0.318113,-0.472892,1.000000,-0.076109,0.078590
air_pressure,0.138620,-0.215872,0.073936,-0.082002,-0.076109,1.000000,-0.029121
wind_speed,0.187672,0.053095,-0.116751,-0.205659,0.078590,-0.029121,1.000000


As shown above, `air_pressure` has little to no relationship with the other features. Also, given its low range described in Table 1, it will be removed. Also, as `humidity` has moderate correlation with `track_temp` and `temp`, it will also be removed. Given we also have a `rain` feature.

In [12]:
df = df.drop(columns=['air_pressure', 'humidity'])

`temp` and `track_temp` also demonstrate a correlation, however, it is unsure which of these features will produce a more accurate model. Therefore, both will be kept and looked at later in the model section of the report.

Now specifically investigating the target features of our report.

In [13]:
df[['fastestLapTime', 'fastestLapSpeed']].describe()

,fastestLapTime,fastestLapSpeed
count,215.000000,215.000000
mean,92600.561456,201.440167
std,11098.117244,19.865508
min,72460.200000,144.650000
25%,83092.395500,191.745000
50%,92599.222000,200.785000
75%,100946.418000,213.594500
max,121779.435000,248.377000


On first glance, range of both `fastestLapTime` and `fastestLapSpeed` seems suitable.

### Categorical Features

We can now safely remove the `data` and `weather` columns as year sufficiently describes date, and rain describes the track conditions.

In [14]:
df = df.drop(columns=['date', 'weather'])

Now lets investigate the categorical features of our dataframe.

In [15]:
categorical_columns = df.columns[df.dtypes=='category'].tolist()
categorical_columns += df.columns[df.dtypes==object].tolist()

for col in categorical_columns:
    print('Unique values for ' + col)
    print(df[col].unique())
    print('')

Unique values for track
[albert-park, sepang, shanghai, sachir, catalunya, ..., montreal, magny-cours, indianapolis, austin, mexico]
Length: 20
Categories (20, object): [albert-park, sepang, shanghai, sachir, ..., magny-cours, indianapolis, austin, mexico]

Unique values for rain
[dry, wet]
Categories (2, object): [dry, wet]

Unique values for year
[2009, 2008, 2007, 2010, 2011, ..., 2015, 2016, 2017, 2018, 2019]
Length: 13
Categories (13, int64): [2009, 2008, 2007, 2010, ..., 2016, 2017, 2018, 2019]

Unique values for wind_direction
['south' 'turning' 'east' 'southeast' 'northeast' 'northwest' 'west'
 'north' 'southern' 'southwest' 'turning east' 'turning south'
 'western liqueur' 'turning west' 'turning southeast' 'changing']



As shown above, the unique values for `weather` highlights something. Due to a translator function being used in the scrape weather script mentioned earlier, there is some conflicting and similar information being shown. With this in mind, it is likely that this data is not going to aid our models. Therefore, it will be removed.

In [16]:
df = df.drop(columns='wind_direction')

## Statistical Modelling & Performance Evaluation <a name="model">

In [32]:
X = df[['track_temp', 'rain', 'wind_speed', 'track', 'year']]
Y = df['fastestLapSpeed']

# convert categorical into dummy/indicator variables
X = pd.get_dummies(data=X, drop_first=True)

# with sklearn
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
 
# prediction with sklearn
track_temp = 21.0
rain = 'dry'
wind_speed = 4.5
track = 'albert-park'
year = '2016'
#print ('Predicted Fastest Lap Time: \n', regr.predict([[track_temp, rain, wind_speed, track, year]]))

# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()
#predictions = model.predict(X) 
 
print(model.summary())

Intercept: 
 190.753887530575
Coefficients: 
 [  0.07350337  -0.1029107   -7.81970673  19.2837513    3.43426584
   1.18571147 -38.65493628  34.58449735  19.76428126  -5.41245091
   3.43426584  13.30774222  22.70395434  17.67529698  -5.5273481
   8.88557099  47.74276103   7.58294154   8.95018326   3.01199248
  29.41262138  -0.98173158   1.74926434   2.78183977  -1.18428495
  -2.68984302  -2.85253623  -1.15533353  -7.53890134  -5.83627003
  -3.08810457   3.99853002   5.85031211   6.24385538]
                            OLS Regression Results                            
Dep. Variable:        fastestLapSpeed   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     94.97
Date:                Thu, 22 Oct 2020   Prob (F-statistic):           1.71e-97
Time:                        22:12:37   Log-Likelihood:                -631.02
No. Observations:              

#### Analysis of Model
Year not much of a predictor, 
however, 2014 - 2015 (P < 0.1) statistically significant at the 10% level
may indicate fastest lap times in 2014, 2015 were significantly higher than base year of 2007

In [33]:
residuals_full = pd.DataFrame({'actual': df['fastestLapSpeed'], 
                            'predicted': model.fittedvalues, 
                            'residual': model.resid})
residuals_full.head(15)

,actual,predicted,residual
0,214.919,214.289516,0.629484
1,202.321,196.503758,5.817242
2,171.060,189.587976,-18.527976
3,204.151,204.308691,-0.157691
4,199.642,197.602759,2.039241
5,157.768,157.887070,-0.119070
6,217.023,219.304745,-2.281745
7,226.025,224.646256,1.378744
8,190.097,190.534139,-0.437139
9,195.778,195.927791,-0.149791


## Summary & Conclusion <a name="conc">